<a href="https://colab.research.google.com/github/Huy1902/HAC/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
from IPython.display import display
import csv

In [2]:
path_to_data = 'ml-1m/ratings.dat'
path_to_item = 'ml-1m/movies.dat'
path_to_users = 'ml-1m/users.dat'
path_to_dataset = 'ml-1m'
path_to_output = "Hyper-Actor-Critic-for-Recommendation/dataset/ml1m"
seed = 2025

## Extract train and test dataset

In [3]:
data = pd.read_csv(path_to_data, sep='::', 
                    names=['userId', 'itemId', 'rating', 'timestamp'])
data

C:\Users\lymst\AppData\Local\Temp\ipykernel_12476\594470988.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(path_to_data, sep='::',


In [7]:


data['user_click'] = data['rating'].apply(lambda x: 1 if x > 3 else 0)
data = data.sort_values(by=['userId', 'timestamp']).reset_index(drop=True)
result = []
test_df = []
train_df = []
for user_id, user_data in data.groupby('userId'):
    user_data = user_data.reset_index(drop=True)
    click_history = []
    seq_id = 0
    sep = random.randint(int(0.7 * len(user_data)), int(0.9 * len(user_data))) // 10
    # Split into segments of 10 items
    for i in range(0, len(user_data), 10):
        if(i + 10 > len(user_data)):
            break
        segment = user_data[i:i+10]
        slate_of_items = segment['itemId'].tolist()
        user_clicks = segment['user_click'].tolist()
        
        row = {
          'user_id': user_id,
          'slate_of_items': slate_of_items,
          'user_mid': user_clicks,
          'user_mid_history': click_history[:],
          'sequence_id': seq_id
        }
        result.append(row)
        if seq_id < sep:
            train_df.append(row)
        else:
            test_df.append(row)
        # Update click history
        click_history.extend(segment[segment['user_click'] == 1]['itemId'].tolist())
        seq_id += 1

train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
result_df = pd.DataFrame(result)
os.makedirs(path_to_output, exist_ok=True)
train_df.to_csv(os.path.join(path_to_output, 'train.csv') , sep='@', index=False)
test_df.to_csv(os.path.join(path_to_output, 'test.csv') , sep='@', index=False)

result_df.to_csv(os.path.join(path_to_output, 'all.csv') , sep='@', index=False)

## Extract item_info

In [12]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76920 entries, 0 to 76919
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           76920 non-null  int64 
 1   slate_of_items    76920 non-null  object
 2   user_mid          76920 non-null  object
 3   user_mid_history  76920 non-null  object
 4   sequence_id       76920 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20463 entries, 0 to 20462
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           20463 non-null  int64 
 1   slate_of_items    20463 non-null  object
 2   user_mid          20463 non-null  object
 3   user_mid_history  20463 non-null  object
 4   sequence_id       20463 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 799.5+ KB


In [7]:
import numpy as np
import os

# Define the order of genres as per the problem statement
genre_order = [
    'Action',
    'Adventure',
    'Animation',
    "Children's",
    'Comedy',
    'Crime',
    'Documentary',
    'Drama',
    'Fantasy',
    'Film-Noir',
    'Horror',
    'Musical',
    'Mystery',
    'Romance',
    'Sci-Fi',
    'Thriller',
    'War',
    'Western'
]

# Initialize genre_features with a dummy row for item 0
genre_features = []
genre_features.append([0] * len(genre_order))  # Dummy row for index 0

# Path to the input movies.dat file and output directory
path_to_movies = path_to_item
path_to_output = path_to_output

# Read the movies.dat file and process each line
with open(path_to_movies, "r", encoding="latin-1") as f:
    for line in f:
        # Split the line into MovieID, Title, Genres
        parts = line.strip().split('::')
        if len(parts) != 3:
            continue  # Skip malformed lines
        _, _, genres_str = parts
        # Split genres into a list
        movie_genres = genres_str.split('|')
        # Create a binary vector for genres
        genre_vector = [0] * len(genre_order)
        for genre in movie_genres:
            if genre in genre_order:
                idx = genre_order.index(genre)
                genre_vector[idx] = 1
        genre_features.append(genre_vector)

# Convert the list to a NumPy array
item_info = np.array(genre_features, dtype=np.float32)

# Save the array to item_info.npy
np.save(os.path.join(path_to_output, "item_info.npy"), item_info)

In [5]:
item_meta = np.load(os.path.join(path_to_output, "item_info.npy"))
item_meta.shape

(3884, 18)

## Extract user_info

In [7]:
import numpy as np
import os

# Define age categories and their order
age_categories = [1, 18, 25, 35, 45, 50, 56]  # 7 categories

# Initialize list to store user data
users = []

with open(path_to_users, "r", encoding="latin-1") as f:
    for line in f:
        parts = line.strip().split("::")
        if len(parts) != 5:
            continue  # Skip malformed lines
        user_id = int(parts[0])
        gender = parts[1]
        age_code = int(parts[2])
        occupation_code = int(parts[3])
        users.append((user_id, gender, age_code, occupation_code))

# Sort users by UserID
users_sorted = sorted(users, key=lambda x: x[0])

# Process each user into a feature vector
feature_vectors = []
for user in users_sorted:
    _, gender, age_code, occupation_code = user
    
    # One-hot encode age (7 dimensions)
    age_encoding = [0] * len(age_categories)
    if age_code in age_categories:
        idx = age_categories.index(age_code)
        age_encoding[idx] = 1
    
    # One-hot encode gender (2 dimensions)
    gender_encoding = [1, 0] if gender == "M" else [0, 1]
    
    # One-hot encode occupation (21 dimensions)
    occupation_encoding = [0] * 21
    occupation_encoding[occupation_code] = 1  # Directly use occupation_code
    
    # Combine features
    feature_vector = age_encoding + gender_encoding + occupation_encoding
    feature_vectors.append(feature_vector)

# Add padding row for user_id=0
padding_row = np.zeros((1, len(feature_vectors[0])), dtype=np.float32)
user_features = np.array(feature_vectors, dtype=np.float32)
user_info = np.concatenate((padding_row, user_features), axis=0)

# Save to user_info.npy
np.save(os.path.join(path_to_output, "user_info.npy"), user_info)

In [8]:
user_meta = np.load(os.path.join(path_to_output, "user_info.npy"))
user_meta.shape

(6041, 30)

## Data extraction and Loading

In [5]:
class DataGenerator():

    def __init__ (self):
        path_to_data = 'ml-100k/u.data'
        path_to_item = 'ml-100k/u.item'
        self.mr = self.load(path_to_data, path_to_item)
        # print(self.mr.info())
        self.users = self.mr['userId'].unique()
        self.movie = self.mr['itemId'].unique()
        self.hist = self.generate_history_function()
        
        self.train = []
        self.test = []

    def load(self, path_to_data, path_to_item):
        data = pd.read_csv(path_to_data, sep='\t', 
                            names=['userId', 'itemId', 'rating', 'timestamp'])
        movie_titles = pd.read_csv(path_to_item, sep='|', names=['itemId', 'itemName'],
                                usecols=range(2), encoding='latin-1')
        
        return data.merge(movie_titles, on='itemId', how='left')
    def generate_history_function(self):
        '''
        Group all rates given by users ad store them from older to most recent based on time
        '''
        historic_users = []
        for i, u in enumerate(self.users):
            movie_watched = self.mr[self.mr['userId'] == u]
            movie_watched = movie_watched.sort_values('timestamp').reset_index(drop=True)
            historic_users.append(movie_watched)

        return historic_users
    def sample_history(self, user_hist, action_ratio=0.8, max_samp_by_user=5, max_state=100,
                       max_action=50, n_states=[], n_actions=[]):
        '''
        Making multiple samples for a given historic set, based on some number of states and actions
        '''

        n = len(user_hist)
        sep = int(action_ratio * n)
        n_sample = random.randint(1, max_samp_by_user)
        if not n_states:
            n_states = [min(random.randint(1, sep), max_state) for _ in range(n_sample)]
        if not n_actions:
            n_actions = [min(random.randint(1, n - sep), max_action) for _ in range(n_sample)]

        assert len(n_states) == len(n_actions), 'Must have same size'

        states = []
        actions = []

        for i in range(len(n_states)):
            sample_states = user_hist.iloc[0:sep].sample(n_states[i])
            sample_actions = user_hist.iloc[-(n-sep):].sample(n_actions[i])
            
            # format state and action
            sample_state = []
            sample_action = []
            for index in range(n_states[i]):
                row = sample_states.iloc[index]
                state = str(row.loc['itemId']) + '&' + str(row.loc['rating'])
                sample_state.append(state)
                
            for index in range(n_actions[i]):
                row = sample_actions.iloc[index]
                action = str(row.loc['itemId']) + '&' + str(row.loc['rating'])
                sample_action.append(action)
                
            states.append(sample_state)
            actions.append(sample_action)
        return states, actions
    
    def get_train_test_split(self, test_ratio, seed=None):
        '''
        Split the data into train and test set
        '''
        if seed:
            random.Random(seed).shuffle(self.hist)
        else:
            random.shuffle(self.hist)
            
        sep = int(test_ratio * len(self.hist))
        self.train = self.hist[:sep]
        self.test = self.hist[sep:]
        self.user_train = [h.iloc[0, 0] for h in self.train]
        self.user_test = [h.iloc[0, 0] for h in self.test]
        
    def write_csv(self, filename, hist_to_write, delimiter=';', action_ratio=0.8, max_sample_user=5,
                  max_state=100, max_action=50, n_states=[], n_actions=[]):
        with open(filename, mode='w') as file:
            f_writer = csv.writer(file, delimiter=delimiter)
            f_writer.writerow(['state', 'action_reward', 'n_state'])
            for user_hist in hist_to_write:
                states, actions = self.sample_history(user_hist, action_ratio, max_sample_user, max_state, max_action, n_states, n_actions)
                for i in range(len(states)):
                    state_str = '|'.join(states[i])
                    action_str = '|'.join(actions[i])
                    n_state_str = state_str + '|' + action_str
                    f_writer.writerow([state_str, action_str, n_state_str])


In [12]:
class EmbeddingModel(nn.Module):
  def __init__ (self, movie_count, hidden_layer_size=100):
    super(EmbeddingModel, self).__init__()
    self.movie_count = movie_count
    self.hidden_layer = nn.Linear(movie_count, hidden_layer_size)
    self.dropout = nn.Dropout(0.2)
    self.output_layer = nn.Linear(hidden_layer_size, movie_count)
    
  def forward(self, x):
    x = self.hidden_layer(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.output_layer(x)
    x = F.softmax(x, dim=-1)
    return x
movie_count = 500  # Adjust based on your dataset
model = EmbeddingModel(movie_count)
print(model)

EmbeddingModel(
  (hidden_layer): Linear(in_features=500, out_features=100, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (output_layer): Linear(in_features=100, out_features=500, bias=True)
)


In [8]:
class EmbeddingGenerator:

    def __init__(self, train_users, data):
      self.train_users = train_users
      
      self.data = data.sort_values(by=['timestamp'])
      self.data['userId'] = self.data['userId'] - 1
      self.data['itemId'] = self.data['itemId'] - 1
      
      self.user_count = self.data['userId'].max() + 1
      self.movie_count = self.data['itemId'].max() + 1
      
      # list of rated movies by each user
      self.user_movies = {}
      for userId in range(self.user_count):
        self.user_movies[userId] = self.data[self.data.userId == userId].itemId.tolist()
      self.model = self.define_model()
      
    def define_model(self, hidden_layer_size=100):
      

_IncompleteInputError: incomplete input (3085494191.py, line 20)